# Used Functions

In [1]:
def fixed_point_to_binary(fixed_point_data, num_bits, fractional_bits):
    # Calculate the maximum integer value that can be represented with the given number of bits
    max_int_value = 2**(num_bits - 1) - 1
    
    # Scale the fixed-point data to fit within the representable integer range
    scaled_data = int(fixed_point_data * (2**fractional_bits))
    
    # Clip the scaled data to stay within the representable integer range
    clipped_data = max(-max_int_value, min(scaled_data, max_int_value))
    
    # Convert the clipped data to binary representation
    binary_representation = bin(clipped_data & int("1"*num_bits, 2))[2:].zfill(num_bits)
    
    return binary_representation

# Base Video Pipeline

## Define input parameters

In [2]:
scaling = 0.01
fixed_format = [16, 8]

fixed_scaling = fixed_point_to_binary(scaling, fixed_format[0], fixed_format[1])
int_version = int(fixed_scaling, 2)

print(f'Scaling factor: {scaling}')
print(f'Adopted Fixed-Point Format: ap_ufixed<{fixed_format[0]},{fixed_format[1]}>')
print(f'Integer Representation: {int_version}')

Scaling factor: 0.01
Adopted Fixed-Point Format: ap_ufixed<16,8>
Integer Representation: 2


## Pipeline

Load overlay

In [ ]:
from pynq import Overlay
from pynq.lib.video import *

base = Overlay('base.bit')
base.download()

hdmi_in = base.video.hdmi_in
hdmi_out = base.video.hdmi_out

Configure HDMI

In [ ]:
hdmi_in.configure(PIXEL_RGBA)
hdmi_out.configure(hdmi_in.mode, PIXEL_RGBA)

hdmi_in.start()
hdmi_out.start()

hdmi_in.tie(hdmi_out)

Write configuration data (for the `invstripe` function)

In [ ]:
from pynq import MMIO
invstripe = MMIO(0x43C8_0000, 0x10000)
invstripe.write(0x10, int_version)

Don't forget to close the HDMI connection!

In [ ]:
hdmi_out.close()
hdmi_in.close()